In [14]:
import struct
import chainparams
import hashlib


class Message:

    def __init__(self, command, payload):
        self.magic = struct.pack('>I', chainparams.mainParams.StartString)
        self.command = command
        self.command_bytes = None
        self.length = struct.pack('<I', len(payload))
        self.payload = payload
        self.checksum = None
        self.header = None
        
    def serialize_payload(self):
        if not isinstance(self.payload, bytes):
            self.payload = self.payload.encode('utf-8')
        self.length = struct.pack('<I', len(self.payload))

        double_hash = hashlib.sha256(hashlib.sha256(self.payload).digest()).digest()
        self.checksum = struct.pack('<4s', double_hash[:4])

    def generate_header(self):
        self.serialize_payload()

        # serialize and pack command message
        b = self.command.encode('utf-8')
        self.command_bytes = struct.pack('<12s', b)

        # Create the whole header
        self.header = b"".join([self.magic, self.command_bytes, self.length, self.checksum])
        return self.header

    def to_bytes(self):
        self.generate_header()
        msg = b"".join([self.header, self.payload])
        return msg


class Verack(Message):

    def __init__(self):
        self.command = "verack"
        self.payload = "hello, world"
        Message.__init__(self, command=self.command, payload=self.payload)
        



In [15]:
m = Message('version', 'hello, world')
print(vars(m))
v = Verack()
print(vars(v))
v.to_bytes()

{'magic': b'\xf9\xbe\xb4\xd9', 'command': 'version', 'command_bytes': None, 'length': b'\x0c\x00\x00\x00', 'payload': 'hello, world', 'checksum': None, 'header': None}
{'command': 'verack', 'payload': 'hello, world', 'magic': b'\xf9\xbe\xb4\xd9', 'command_bytes': None, 'length': b'\x0c\x00\x00\x00', 'checksum': None, 'header': None}


b'\xf9\xbe\xb4\xd9verack\x00\x00\x00\x00\x00\x00\x0c\x00\x00\x00B-lUhello, world'